In [1]:
import numpy as np
import cv2

webcam = True
phoneCamera = 'https://192.168.1.46:8080/'
path = 'C:/Users/Acer/Desktop/jupyter/test.png'
cap = cv2.VideoCapture('https://192.168.1.46:8080/video' ) # This is the ip address of my Phone camera.
scale = 3
width_of_paper = 210 * scale
height_of_paper = 297 * scale #in milimeters. this is for A4 paper


cap.set(10, 160) 
cap.set(3, 1920) #width
cap.set(4, 1080) #height


#Canny Edge Detection Function. 
def getContours(img, cThr=[100, 100], showCanny=False, minArea=1000, filter=0, draw = False):
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    imgBlur = cv2.GaussianBlur(imgGray,(5,5),1)
    imgCanny = cv2.Canny(imgBlur, cThr[0], cThr[1])
    kernel = np.ones((5, 5))
    imgDial = cv2.dilate(imgCanny, kernel, iterations = 3)
    imgThreshold = cv2.erode(imgDial, kernel, iterations = 2)
    if showCanny:
        cv2.imshow('Canny', imgThreshold)

    contours, hierarchy = cv2.findContours(imgThreshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    #loop through find areas, draw them, and see how well is our detection
    
    finalContours = []
    
    for i in contours:
        area = cv2.contourArea(i)
        if area > minArea:
            perimeter = cv2.arcLength(i, True)
            approx = cv2.approxPolyDP(i, 0.02 * perimeter, True)
            boundingBox = cv2.boundingRect(approx) 

            #if we only want to measure rectangles. change if ever we will switch shapes
            if filter > 0:
                if len(approx) == filter:
                    finalContours.append([len(approx), area, approx, boundingBox, i])
            else:
                finalContours.append([len(approx), area, approx, boundingBox, i])
    #sort contours based on size
    finalContours = sorted(finalContours, key = lambda x:x[1], reverse = True)
    #draw the contours
    if draw:
        for con in finalContours:
            cv2.drawContours(img, con[4],-1,(0,0,255),3)
    return img, finalContours

# Warping Function
def reorder(myPoints):
    print(myPoints.shape)
    myPointsNew = np.zeros_like(myPoints)
    myPoints = myPoints.reshape((4,2))
    add = myPoints.sum(1)
    myPointsNew[0] = myPoints[np.argmin(add)]
    myPointsNew[3] = myPoints[np.argmax(add)]
    diff = np.diff(myPoints, axis=1)
    myPointsNew[1] = myPoints[np.argmin(diff)]
    myPointsNew[2] = myPoints[np.argmax(diff)]
    return myPointsNew

def warpImg(img, points,w,h,pad=20):
    #print(points)
    points = (reorder(points))

    pts1 = np.float32(points)
    pts2 = np.float32([[0,0], [w,0], [0,h], [w,h]])
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    imgWarp = cv2.warpPerspective(img, matrix, (w,h))
    imgWarp = imgWarp[pad:imgWarp.shape[0]-pad,pad:imgWarp.shape[1]-pad] 

    return imgWarp

#now that we have drawn the lines, and have identified the points 1,2,3,4,

#the last step to calculate the lengths to give values is to setup an algorithm to use Pythagorean theorem, 

def findDis(pts1, pts2):
    return ((pts2[0]-pts1[0])**2 + (pts2[1]-pts1[1])**2)**0.5 #this function returns the distance and magnitude of lines

In [2]:
#run Full Object-Measuring

while True:
    if webcam:
        success, img = cap.read() 
    else: 
        img = cv2.imread(path)
    imgContours, conts = getContours(img, minArea=50000, filter=4) #filter = 4 points, only measures objects with 4 corners!
    if len(conts) != 0:
        largest = conts[0][2]
        imgWarp = warpImg(img, largest, width_of_paper, height_of_paper )
        imgContours2, conts2 = getContours(imgWarp, minArea=2000, filter=4, cThr=[50,50], draw = False)

        if len(conts2) != 0:
            for obj in conts2:
                cv2.polylines(imgContours2, [obj[2]], True,(0,255,0), 1)
                nPoints = reorder(obj[2])
                nW = round((findDis( nPoints[0][0]//scale,nPoints[1][0]//scale) /10),1)
                nH = round((findDis( nPoints[0][0]//scale,nPoints[2][0]//scale) /10),1)
                #code here just displays the value, of what is calculated above
                cv2.arrowedLine(imgContours2, (nPoints[0][0][0], nPoints[0][0][1]), (nPoints[1][0][0], nPoints[1][0][1]),(255, 0, 255), 3, 8, 0, 0.05)
                cv2.arrowedLine(imgContours2, (nPoints[0][0][0], nPoints[0][0][1]), (nPoints[2][0][0], nPoints[2][0][1]),(255, 0, 255), 3, 8, 0, 0.05)
                x, y, w, h = obj[3]
                cv2.putText(imgContours2, '{}cm'.format(nW), (x + 30, y - 10),cv2.FONT_HERSHEY_COMPLEX_SMALL, 1.5,(255, 0, 255), 2)
                cv2.putText(imgContours2, '{}cm'.format(nH), (x - 70, y + h // 2), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1.5,(255, 0, 255), 2)

                ###############################################################
        
        cv2.imshow('A4 Paper', imgContours2)
    
    img = cv2.resize(img, (0,0), None, 0.5, 0.5)
    getContours(img, showCanny = True) # Show the Bounding Box
    cv2.imshow('Original', img)
    cv2.waitKey(1)

(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)
(4, 1, 2)


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
